正常的函数在执行时是不会中断的，所以你要写一个能够中断的函数，就需要添加async关键。

async 用来声明一个函数为异步函数，异步函数的特点是能在函数执行过程中挂起，去执行其他异步函数，等到挂起条件（假设挂起条件是sleep(5)）消失后，也就是5秒到了再回来执行。

await 用来用来声明程序挂起，比如异步程序执行到某一步时需要等待的时间很长，就将此挂起，去执行其他的异步程序。await 后面只能跟异步程序或有__await__属性的对象，因为异步程序与一般程序不同。假设有两个异步函数async a，async b，a中的某一步有await，当程序碰到关键字await b()后，异步程序挂起后去执行另一个异步b程序，就是从函数内部跳出去执行其他函数，**当挂起条件消失后，不管b是否执行完，要马上从b程序中跳出来，回到原程序执行原来的操作**。

如果await后面跟的b函数不是异步函数，那么操作就只能等b执行完再返回，无法在b执行的过程中返回。如果要在b执行完才返回，也就不需要用await关键字了，直接调用b函数就行。所以这就需要await后面跟的是异步函数了。在一个异步函数中，可以不止一次挂起，也就是可以用多个await。

In [1]:
import types

def function():
    """普通函数"""
    return 1

def generator():
    """生成器函数"""
    yield 1

async def async_function():
    """异步函数（协程）"""
    return 1

async def async_generator():
    """异步生成器"""
    yield 1


In [2]:
print(type(function) is types.FunctionType)
print(type(generator()) is types.GeneratorType)
print(type(async_function()) is types.CoroutineType)
print(type(async_generator()) is types.AsyncGeneratorType)

True
True
True
True


C:\Users\Huster2021\AppData\Local\Temp\ipykernel_74868\4000583770.py:3: RuntimeWarning: coroutine 'async_function' was never awaited
  print(type(async_function()) is types.CoroutineType)


In [3]:
print(async_function()) 
# <coroutine object async_function at 0x102ff67d8> 直接调用异步函数返回一个 coroutine 对象
# 协程需要其他方式来驱动 因此可以使用该协程对象的 send 方法给协程发送一个值

<coroutine object async_function at 0x000001877F334540>


C:\Users\Huster2021\AppData\Local\Temp\ipykernel_74868\2846481357.py:1: RuntimeWarning: coroutine 'async_function' was never awaited
  print(async_function())


In [5]:
print(async_function().send(None)) 
#如果通过上面的调用会抛出一个异常： StopIteration: 1
# # 因为生成器/协程在正常返回退出时会抛出一个StopIteration异常，
# 而原来的返回值会存放在StopIteration对象的value属性中，

StopIteration: 1

In [8]:
# 通过以下捕获可以获取协程真正的返回值：
try:
    async_function().send(None)
except StopIteration as e:
    print(e.value)

1


In [9]:
#通过上面的方式来新建一个run函数来驱动协程函数：
def run(coroutine):
    try:
        coroutine.send(None)
    except StopIteration as e:
        return e.value

In [12]:
# 在协程函数中，可以通过await语法来挂起自身的协程，并等待另一个协程完成直到返回结果：
# await语法只能出现在通过async修饰的函数中，否则会报SyntaxError错误。
# 而且await后面的对象需要是一个Awaitable，或者实现了相关的协议
async def async_function():
    return 2

async def await_coroutine():
    result = await async_function()
    print(result)

run(await_coroutine())

2


In [14]:
# 查看Awaitable抽象类的代码，表明了只要一个类实现了__await__方法，那么通过它构造出来的实例就是一个Awaitable：
class Awaitable(metaclass=ABCMeta):
    __slots__ = ()

    @abstractmethod
    def __await__(self):
        yield

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Awaitable:
            return _check_methods(C, "__await__")
        return NotImplemented
# 而且可以看到，Coroutine类也继承了Awaitable，而且实现了send，throw和close方法。
# 所以await一个调用异步函数返回的协程对象是合法的。
class Coroutine(Awaitable):
    __slots__ = ()

    @abstractmethod
    def send(self, value):
        ...

    @abstractmethod
    def throw(self, typ, val=None, tb=None):
        ...

    def close(self):
        ...
        
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Coroutine:
            return _check_methods(C, '__await__', 'send', 'throw', 'close')
        return NotImplemented

NameError: name 'ABCMeta' is not defined

In [17]:
# 一个异步生成器的例子
# 假设要去一家超时买土豆，货架上的土豆是有限的
class Potato:
    # 不需要实例化，不需要 self 参数，但第一个参数需要是表示自身类的 cls 参数，
    # 可以来调用类的属性，类的方法，实例化对象等。
    @classmethod
    def make(cls, num, *args, **kws):
        potatos = []
        for i in range(num):
            potatos.append(cls.__new__(cls,*args,**kws))
        return potatos

all_potatoes = Potato.make(5)

print(len(all_potatoes))
print(all_potatoes)

5
[<__main__.Potato object at 0x000001877F1BE3A0>, <__main__.Potato object at 0x000001877F1BECD0>, <__main__.Potato object at 0x000001877F190EB0>, <__main__.Potato object at 0x000001877F190550>, <__main__.Potato object at 0x000001877F3360A0>]


In [19]:
# 需求：想买 50 个土豆，每次从货架撒花姑娘拿走一个放到土豆篮子里面
def take_potatos(num):
    count = 0
    while True:
        if len(all_potatoes) == 0:
            sleep(.1)
        else:
            potato = all_potatoes.pop()
            yield potato
            count += 1
            if count == num:
                break


def buy_potato():
    bucket = []
    for p in take_potatos(50):
        bucket.append(p)
     

In [21]:
import asyncio
import random

In [22]:
async def ask_for_potato():
    await asyncio.sleep(random.random())
    all_potatoes.extend(Potato.make(random.randint(1,10)))

In [26]:
# 协程中这样用
async def take_potatoes(num):
    count = 0
    while True:
        if len(all_potatoes) == 0:
            # 货架上没有土豆之后，询问超市请求更多的土豆
            await ask_for_potato()
        potato = all_potatoes.pop()
        yield potato
        count += 1
        if count == num:
            break

In [27]:
async def buy_potatoes():
    bucket = []
    async for p in take_potatoes(50):
        bucket.append(p)
        print(f'Got potato {id(p)}...')

In [28]:
async def buy_tomatoes():
    bucket = []
    async for p in take_potatoes(50):
        bucket.append(p)
        print(f'Got potato {id(p)}...')

In [29]:
def main():
    import asyncio
    loop = asyncio.get_event_loop()
    res = loop.run_until_complete(asyncio.wait([buy_potatoes(), buy_tomatoes()]))
    loop.close()